In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"s3://vasveena-test-demo/jars/iceberg-spark3-runtime-0.11.1.jar",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
             "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
             "spark.sql.catalog.spark_catalog.type":"hive",
             "spark.sql.catalog.local":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.local.type":"hadoop",
             "spark.sql.catalog.local.warehouse":"s3://vasveena-test-demo/iceberg/catalog/tables/"
           } 
}

In [2]:
val input_df = spark.read.parquet("s3://neilawstmp2/tmp/hudi-perf/input/")
input_df.count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1629139174081_0036,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 5 more fields]
res1: Long = 100000000


In [3]:
val input_df2=(input_df.withColumn("z", substring(md5(concat($"id")),1,1))
.withColumn("schema-v", lit("v1")).withColumn("data-v", lit("v2"))
.withColumn("trade_dt", substring($"modified_timestamp",1,10)))
input_df2.show(5)

val input_df4 = input_df2.withColumnRenamed("schema-v", "schema_v").withColumnRenamed("data-v", "data_v")

input_df4.registerTempTable("inputdf4")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

input_df2: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|[E]|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|[F]|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|[G]|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|[H]|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|[I]|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+--------------------+----+-----------------

In [9]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf26 (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.target-file-size-bytes'=20971520,
          'write.parquet.row-group-size-bytes'=20971520,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf26'""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res18: org.apache.spark.sql.DataFrame = []


In [8]:
spark.sql("drop table local.db.iceberg_table_sparksqldf26")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.sql.catalyst.analysis.NoSuchTableException: Table db.iceberg_table_sparksqldf26 not found;
  at org.apache.spark.sql.execution.datasources.v2.DropTableExec.run(DropTableExec.scala:35)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:39)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:39)
  at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:45)
  at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:230)
  at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3667)
  at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:104)
  at org.apache.spark.sql.execution.SQLExecution$.withTracker(SQLExecution.scala:227)
  at org.apache.spark.sql.execution.SQLExecution$.executeQuery$1(SQLExecution.scala:107)
  at org.apache.spark.sql.execution.SQ

In [6]:
val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf26")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_parqrowgroupsizeandtargetsize for this test where both
// write.target-file-size-bytes and write.parquet.row-group-size-bytes were set

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 305607782109057
duration: String = 130.181471962seconds


In [32]:
import org.apache.spark.sql.functions._
val df = spark.table("local.db.iceberg_table_sparksqldf26")
df.agg(countDistinct("z", "schema_v","data_v","trade_dt").as("distinctCounts")).show(false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
df: org.apache.spark.sql.DataFrame = [id: bigint, month: bigint ... 9 more fields]
+--------------+
|distinctCounts|
+--------------+
|16            |
+--------------+



In [50]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf22 (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          --'write.target-file-size-bytes'=20971520,
          --'write.parquet.row-group-size-bytes'=20971520,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf22'""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res85: org.apache.spark.sql.DataFrame = []


In [37]:
val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf22")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_default for this test where 
//neither write.target-file-size-bytes nor write.parquet.row-group-size-bytes were set

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 299631140646338
duration: String = 117.399936972seconds


In [49]:
spark.sql("drop table local.db.iceberg_table_sparksqldf22")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res84: org.apache.spark.sql.DataFrame = []


In [38]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf24 (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          --'write.target-file-size-bytes'=20971520,
          'write.parquet.row-group-size-bytes'=20971520,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf24'""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res56: org.apache.spark.sql.DataFrame = []


In [39]:
val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf24")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_parqrowgroupsizeonly for this test setting only write.parquet.row-group-size-bytes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 299957707048795
duration: String = 113.369165187seconds


In [40]:
spark.sql("""CREATE TABLE local.db.iceberg_table_sparksqldf28 (id bigint,
                                       month bigint,
                                       sk bigint,
                                       txt struct<key1:string>,
                                       uuid string,
                                       year string,
                                       modified_timestamp timestamp,
                                       z string,
                                       schema_v string,
                                       data_v string,
                                       trade_dt string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.target-file-size-bytes'=20971520,
          --'write.parquet.row-group-size-bytes'=20971520,
          'write.object-storage.path'='s3://vasveena-test-hmswh/')
PARTITIONED BY (z,schema_v,data_v,trade_dt)
location  's3://vasveena-test-demo/iceberg/catalog/tables/db/iceberg_table_sparksqldf28'""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res61: org.apache.spark.sql.DataFrame = []


In [41]:
val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf28")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_onlytargetsize for this test setting only write.target-file-size-bytes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 300435418324943
duration: String = 116.956403467seconds


In [44]:
//dropped and recreated table local.db.iceberg_table_sparksqldf22 where neither of the target file size properties were set 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
//insert with repartition 
val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").repartition(100).write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf22")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_repartition100


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 302051274772223
duration: String = 111.910471878seconds


In [52]:
//dropped and recreated table local.db.iceberg_table_sparksqldf26 where both target file size properties were set 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
//re-inserting with repartition

val t1 = System.nanoTime

//order by clause is needed to avoid error "Caused by: java.lang.IllegalStateException: Already closed files for partition: z=c/schema_v=v1/data_v=v2/trade_dt=2021-04-02"
input_df4.orderBy("z","`schema-v`","`data-v`","trade_dt").repartition(100).write.mode("overwrite").insertInto("local.db.iceberg_table_sparksqldf26")

val duration = (System.nanoTime - t1) / 1e9d + "seconds"

//check S3 listing in file hmswhl_repartition100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t1: Long = 305963713905968
duration: String = 121.303479111seconds


In [2]:
spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1633315542313_0024,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 3.0.1-amzn-0


In [5]:
spark.sql("delete from local.db.iceberg_table_sparksqldf24 where z=1 and schema_v=v1 and data_v=v2 and trade_dt=2021-04-04")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.sql.catalyst.parser.ParseException:
extraneous input 'PURGE' expecting {<EOF>, ';'}(line 1, pos 112)

== SQL ==
delete from local.db.iceberg_table_sparksqldf24 where z=1 and schema_v=v1 and data_v=v2 and trade_dt=2021-04-04 PURGE
----------------------------------------------------------------------------------------------------------------^^^

  at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:266)
  at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:133)
  at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:48)
  at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:81)
  at org.apache.spark.sql.catalyst.parser.extensions.IcebergSparkSqlExtensionsParser.parsePlan(IcebergSparkSqlExtensionsParser.scala:100)
  at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:605)
  at org.apache.spark.sql.catalyst.Quer

In [8]:
spark.sql("alter table local.db.iceberg_table_sparksqldf24 drop partition field (z='1', schema_v='v1', data_v='v2', trade_dt='2021-04-02') PURGE")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.sql.catalyst.parser.ParseException:
extraneous input '(' expecting {'ADD', 'ALTER', 'AS', 'ASC', 'BY', 'CALL', 'DESC', 'DROP', 'FIELD', 'FIRST', 'LAST', 'NULLS', 'ORDERED', 'PARTITION', 'TABLE', 'WRITE', 'TRUE', 'FALSE', 'MAP', IDENTIFIER, BACKQUOTED_IDENTIFIER}(line 1, pos 69)

== SQL ==
alter table local.db.iceberg_table_sparksqldf24 drop partition field (z='1', schema_v='v1', data_v='v2', trade_dt='2021-04-02') PURGE
---------------------------------------------------------------------^^^

  at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:266)
  at org.apache.spark.sql.catalyst.parser.extensions.IcebergSparkSqlExtensionsParser.parse(IcebergSparkSqlExtensionsParser.scala:145)
  at org.apache.spark.sql.catalyst.parser.extensions.IcebergSparkSqlExtensionsParser.parsePlan(IcebergSparkSqlExtensionsParser.scala:98)
  at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:605)
  at org.apache.sp

In [ ]:
ALTER TABLE prod.db.sample DROP PARTITION FIELD shard 